# Puropse of this notebook

This code has been used as part of `phase_notebook.ipynb` to plot a PRC with both EMSI and current signals below it. It doesn't work now, it would need importing more elements from the phase notebook.

In [2]:
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks
from matplotlib import pyplot as plt

%matplotlib qt

In [ ]:
def read_datfile(filename, p_height=0.5, margins=(100, 5000)):
    
    df = pd.read_csv(filename, sep='\t', header=5)
    
    
    # Renaming columns
    new_cols = ['t', 'U', 'I', 'Ill', 'EMSI']
    old_cols = df.columns
    col_dict = {x: y for x, y in zip(old_cols, new_cols)}
    df = df.rename(columns = col_dict)
    
    # Flipping signs
    df['U'] = -df['U']
    df['I'] = -df['I']
    
    # Applying margins
    df = df[(df['t'] > margins[0]) & (df['t'] < margins[1])]
    
    # Signal smoothening
    df['I'] = gaussian_filter(df['I'], 10)*100_000
    df['EMSI'] = gaussian_filter(df['EMSI'], 2)
    # df['U'] = gaussian_filter(df['U'], 1)
    
    # Finding voltage spikes
    spike_indicies, _ = find_peaks(np.sign(p_height) * df['U'],
                                   prominence=np.abs(p_height)*0.8, distance=1000)
    spikes = df.iloc[spike_indicies]
    
    return df, np.array(spikes['t'])


In [ ]:
df, _ = read_datfile('~/munich/data/24X/A00102.dat')

crossings = df[(np.diff(np.sign(df['I']-1.1*np.mean(df['I'])), append=0) == 2)]
crossings = np.array(crossings['t'])
which_period = 10
one_period = df[(df['t']>crossings[which_period]) & (df['t']<crossings[which_period+1])]
one_period = one_period.assign(phi= (one_period['t']-crossings[which_period]) / (crossings[which_period+1] - crossings[which_period]))


# Plot PRC
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, sharex=True)
ax1.set_title("PRC -- 1s, +0.3V")

#ax1.set_xlabel(r'$\phi$ (fractional)')
ax1.set_ylabel('Phase response')
sc = ax1.scatter(perts['phase'], -perts['response']/perts['expected_period'], c=perts['time']/3600)
# cbar = plt.colorbar(sc)
# cbar.set_label('time [h]')
ax1.set_xlim(0, 1)
ax1.axhline(0, c='m', ls='--')
plt.tight_layout()

color = 'tab:blue'

ax2.set_xlabel(r'$\phi$ (fractional)')
ax2.set_ylabel('current', color=color)
ax2.plot(one_period['phi'], one_period['I'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

ax3 = ax2.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax3.set_ylabel('EMSI', color=color)  # we already handled the x-label with ax1
ax3.plot(one_period['phi'], one_period['EMSI'], color=color)
ax3.tick_params(axis='y', labelcolor=color)

plt.tight_layout()
plt.show()
# Plot periods vs time
plt.figure()
plt.title('Period length versus time')
plt.scatter(cycles['start'], cycles['duration'], marker='+')
plt.plot(cycles['start'], np.polyval(period_fit, cycles['start']), 'm-')

# for x in pert_times:
    # plt.axvline(x, c='r', ls='--')
plt.title('Perturbation: +0.3V, 1s')
plt.xlabel('Time [s]')
plt.ylabel('Osc. period [s]')
# Plot current vs time
plt.close()
plt.figure()
plt.title('Threshold = mean current')
plt.plot(data['t'], data['I'])
plt.axhline(threshold_I, c='y', ls='dashed')
plt.scatter(cycles['start'], data[data['t'].isin(cycles['start'])]['I'], marker='x', c='r')
for t in cycles['start']:
    plt.axvline(t, c='grey', ls=':')
#plt.scatter(pert_times, data[data['t'].isin(pert_times)]['I'], marker='x', c='r')
plt.xlabel('Time [s]')
plt.ylabel('Current [A]')
# plt.plot(data['t'], data['U'], 'm-')
# for x in cycles['start']:
    # plt.axvline(x, c='g', ls='-.')
plt.xlim(2000, 2100)

In [ ]:
one_period

In [ ]:

spikes, _ = find_peaks(data['I'], height=0.06, distance=1000)
time_array = np.array(data['t'])
spike_times = time_array[spikes]

In [ ]:
plt.plot(cycles['duration'])
plt.plot(cycles['expected_duration'])